# Main model classic double robust DiD

Note: 
- I caluzclate the average Variance not asymptotic Variance. i dont know what that is tbh

## classic Double robust

In [ ]:
import numpy as np
from doubleml import DoubleMLData, DoubleMLDID
from doubleml.datasets import make_did_SZ2020

check the new paper I found and this repository to replicate the results of sant'anna and Zhao (2021) paper. https://github.com/tommaso-manfe/DR-DIPW-Updated-Beyond-Regression-DiD/tree/main

In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression

ml_g = LinearRegression()  # as in the paper, estimators not needed
ml_m = LogisticRegression()  # as in the paper, estimators not needed

In [ ]:
n_obs = 1000
n_rep = 100
ATTE = 0.0

ATTE_estimates = np.full((n_rep), np.nan)
coverage = np.full((n_rep), np.nan)
ci_length = np.full((n_rep), np.nan)
asymptotic_variance = np.full(n_rep, np.nan)


np.random.seed(42)
for i_rep in range(n_rep):
    if (i_rep % int(n_rep / 10)) == 0:
        print(f"Iteration: {i_rep}/{n_rep}")
    dml_data = make_did_SZ2020(n_obs=n_obs, dgp_type=2, cross_sectional_data=False)

    dml_did = DoubleMLDID(dml_data, ml_g=ml_g, ml_m=ml_m, n_folds=5)
    dml_did.fit()

    ATTE_estimates[i_rep] = dml_did.coef.squeeze()
    confint = dml_did.confint(level=0.95)
    coverage[i_rep] = (confint["2.5 %"].iloc[0] <= ATTE) & (
        confint["97.5 %"].iloc[0] >= ATTE
    )
    ci_length[i_rep] = confint["97.5 %"].iloc[0] - confint["2.5 %"].iloc[0]

    summary_df = dml_did.summary
    std_err = summary_df.loc["d", "std err"]
    asymptotic_variance[i_rep] = std_err**2

# Calculate metrics
avg_bias = np.mean(ATTE_estimates - ATTE)
med_bias = np.median(ATTE_estimates - ATTE)
rmse = np.sqrt(np.mean((ATTE_estimates - ATTE) ** 2))
avg_asymptotic_variance = np.mean(asymptotic_variance)
coverage_probability = np.mean(coverage)
avg_ci_length = np.mean(ci_length)

# Print results
print(f"Av. Bias: {avg_bias}")
print(f"Med. Bias: {med_bias}")
print(f"RMSE: {rmse}")
print(f"Asy. V: {avg_asymptotic_variance}")
print(f"Cover: {coverage_probability}")
print(f"CIL: {avg_ci_length}")

## TWFE Simulation

DGP 1

In [ ]:
# DGP1

import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import logistic

np.random.seed(42)  # You can use any integer value as the seed

# Define parameters
n = 1000  # Sample size
Xsi_ps = 0.75  # pscore index
_lambda = 0.5  # Proportion in each period
nboot = 199  # Number of bootstrapped draws

# Define means and standard deviations
mean_z1 = np.exp(0.25 / 2)
sd_z1 = np.sqrt((np.exp(0.25) - 1) * np.exp(0.25))
mean_z2 = 10
sd_z2 = 0.54164
mean_z3 = 0.21887
sd_z3 = 0.04453
mean_z4 = 402
sd_z4 = 56.63891

# Initialize empty lists to store results
ATTE_estimates = []
asymptotic_variance = []
coverage = []
ci_length = []

# Loop for 1000 runs
for _i in range(1000):
    # Generate covariates
    x1 = np.random.normal(0, 1, n)
    x2 = np.random.normal(0, 1, n)
    x3 = np.random.normal(0, 1, n)
    x4 = np.random.normal(0, 1, n)

    z1 = np.exp(x1 / 2)
    z2 = x2 / (1 + np.exp(x1)) + 10
    z3 = (x1 * x3 / 25 + 0.6) ** 3
    z4 = (x1 + x4 + 20) ** 2

    z1 = (z1 - mean_z1) / sd_z1
    z2 = (z2 - mean_z2) / sd_z2
    z3 = (z3 - mean_z3) / sd_z3
    z4 = (z4 - mean_z4) / sd_z4

    x = np.column_stack((x1, x2, x3, x4))
    z = np.column_stack((z1, z2, z3, z4))

    # Propensity score
    pi = logistic.cdf(Xsi_ps * (-z1 + 0.5 * z2 - 0.25 * z3 - 0.1 * z4))
    d = np.random.uniform(size=n) <= pi

    # Generate aux indexes for the potential outcomes
    index_lin = 210 + 27.4 * z1 + 13.7 * (z2 + z3 + z4)
    index_unobs_het = d * (index_lin)
    index_att = 0
    index_trend = 210 + 27.4 * z1 + 13.7 * (z2 + z3 + z4)

    # Generate unobserved heterogeneity
    v = np.random.normal(index_unobs_het, 1)

    # Generate outcomes at time 0 and time 1
    y00 = index_lin + v + np.random.normal(size=n)
    y10 = index_lin + v + np.random.normal(size=n)
    y01 = index_lin + v + np.random.normal(scale=1, size=n) + index_trend
    y11 = index_lin + v + np.random.normal(scale=1, size=n) + index_trend + index_att

    # Generate "T"
    ti_nt = 0.5
    ti_t = 0.5
    ti = d * ti_t + (1 - d) * ti_nt
    post = np.random.uniform(size=n) <= ti

    y = np.where(
        d & post,
        y11,
        np.where(~d & post, y01, np.where(~d & ~post, y00, y10)),
    )

    # Generate id
    id_ = np.repeat(np.arange(1, n + 1), 2)
    time = np.tile([0, 1], n)
    # Put in a long data frame
    dta_long = pd.DataFrame(
        {
            "id": id_,
            "time": time,
            "y": np.tile(y, 2),
            "post": np.tile(post.astype(int), 2),
            "d": np.tile(d.astype(int), 2),
            "x1": np.tile(z1, 2),
            "x2": np.tile(z2, 2),
            "x3": np.tile(z3, 2),
            "x4": np.tile(z4, 2),
        },
    )
    dta_long["post:d"] = dta_long["post"] * dta_long["d"]

    dta_long = dta_long.sort_values(["id", "time"])

    # Perform TWFE estimation
    twfe_i = sm.OLS(
        dta_long["y"],
        sm.add_constant(dta_long[["x1", "x2", "x3", "x4", "post", "d", "post:d"]]),
    ).fit()
    twfe = twfe_i.params["post:d"]

    # Calculate asymptotic variance
    asymptotic_variance.append(twfe_i.cov_params()["post:d"])

    # Bootstrap for confidence intervals
    # (Note: This part is not included in the provided code, you'll need to implement it separately)

    # Append TWFE estimate to the list
    ATTE_estimates.append(twfe)

# Convert lists to arrays for ease of calculation
ATTE_estimates = np.array(ATTE_estimates)
asymptotic_variance = np.array(asymptotic_variance)
asymptotic_variance = asymptotic_variance.reshape(-1, 8)
asymptotic_variance = asymptotic_variance[:, 0]

# Calculate metrics
avg_bias = np.mean(ATTE_estimates - 0)  # Assuming ATTE is 0 as mentioned in the code
med_bias = np.median(ATTE_estimates - 0)  # Assuming ATTE is 0 as mentioned in the code
rmse = np.sqrt(
    np.mean((ATTE_estimates - 0) ** 2),
)  # Assuming ATTE is 0 as mentioned in the code
variance_ATT = np.var(ATTE_estimates)
coverage_probability = np.mean(
    ((ATTE_estimates - 1.96 * np.sqrt(variance_ATT)) <= 0)
    & ((ATTE_estimates + 1.96 * np.sqrt(variance_ATT)) >= 0),
)
avg_ci_length = np.mean(
    2 * 1.96 * np.sqrt(variance_ATT),
)  # Length of 95% confidence interval
average_ATT_estimate = np.mean(ATTE_estimates)

# Print results
print(f"Average ATT estimate across all runs: {average_ATT_estimate}")

print(f"Av. Bias: {avg_bias}")
print(f"Med. Bias: {med_bias}")
print(f"RMSE: {rmse}")
print(f"variance_ATT: {variance_ATT}")
print(f"Cover: {coverage_probability}")
print(f"CIL: {avg_ci_length}")

DGP 2

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import logistic

np.random.seed(42)  # You can use any integer value as the seed

# Sample size
n = 1000
# pscore index (strength of common support)
Xsi_ps = 0.75
# Proportion in each period
_lambda = 0.5
# Number of bootstrapped draws
nboot = 199

# Mean and Std deviation of Z's without truncation
mean_z1 = np.exp(0.25 / 2)
sd_z1 = np.sqrt((np.exp(0.25) - 1) * np.exp(0.25))
mean_z2 = 10
sd_z2 = 0.54164
mean_z3 = 0.21887
sd_z3 = 0.04453
mean_z4 = 402
sd_z4 = 56.63891


# Initialize empty lists to store results
ATTE_estimates = []
asymptotic_variance = []
coverage = []
ci_length = []


for _i in range(1000):
    # Generate covariates
    x1 = np.random.normal(0, 1, n)
    x2 = np.random.normal(0, 1, n)
    x3 = np.random.normal(0, 1, n)
    x4 = np.random.normal(0, 1, n)

    z1 = np.exp(x1 / 2)
    z2 = x2 / (1 + np.exp(x1)) + 10
    z3 = (x1 * x3 / 25 + 0.6) ** 3
    z4 = (x1 + x4 + 20) ** 2

    z1 = (z1 - mean_z1) / sd_z1
    z2 = (z2 - mean_z2) / sd_z2
    z3 = (z3 - mean_z3) / sd_z3
    z4 = (z4 - mean_z4) / sd_z4

    x = np.column_stack((x1, x2, x3, x4))
    z = np.column_stack((z1, z2, z3, z4))

    # Generate treatment groups
    # Propensity score
    pi = logistic.cdf(Xsi_ps * (-x1 + 0.5 * x2 - 0.25 * x3 - 0.1 * x4))
    d = np.random.uniform(size=n) <= pi

    # Generate aux indexes for the potential outcomes
    index_lin = 210 + 27.4 * z1 + 13.7 * (z2 + z3 + z4)

    # Create heterogenenous effects for the ATT, which is set approximately equal to zero
    index_unobs_het = d * (index_lin)
    index_att = 0

    # This is the key for consistency of outcome regression
    index_trend = 210 + 27.4 * z1 + 13.7 * (z2 + z3 + z4)

    # v is the unobserved heterogeneity
    v = np.random.normal(index_unobs_het, 1)

    # Gen realized outcome at time 0
    y00 = index_lin + v + np.random.normal(size=n)
    y10 = index_lin + v + np.random.normal(size=n)

    # Gen outcomes at time 1
    # First let's generate potential outcomes: y_1_potential
    y01 = (
        index_lin + v + np.random.normal(scale=1, size=n) + index_trend
    )  # This is the baseline
    y11 = (
        index_lin + v + np.random.normal(scale=1, size=n) + index_trend + index_att
    )  # This is the baseline

    # Generate "T"
    ti_nt = 0.5
    ti_t = 0.5
    ti = d * ti_t + (1 - d) * ti_nt
    post = np.random.uniform(size=n) <= ti

    y = np.where(
        d & post,
        y11,
        np.where(~d & post, y01, np.where(~d & ~post, y00, y10)),
    )

    # Gen id
    id_ = np.repeat(np.arange(1, n + 1), 2)
    time = np.tile([0, 1], n)

    # Put in a long data frame
    dta_long = pd.DataFrame(
        {
            "id": id_,
            "time": time,
            "y": np.tile(y, 2),
            "post": np.tile(post.astype(int), 2),
            "d": np.tile(d.astype(int), 2),
            "x1": np.tile(z1, 2),
            "x2": np.tile(z2, 2),
            "x3": np.tile(z3, 2),
            "x4": np.tile(z4, 2),
        },
    )
    dta_long = dta_long.sort_values(["id", "time"])

    # Create interaction term 'post:d'
    dta_long["post:d"] = dta_long["post"] * dta_long["d"]

    twfe_i = sm.OLS(
        dta_long["y"],
        sm.add_constant(dta_long[["x1", "x2", "x3", "x4", "post", "d", "post:d"]]),
    ).fit()
    twfe = twfe_i.params["post:d"]

    asymptotic_variance.append(
        twfe_i.HC0_se["post:d"],
    )  # Append the standard error for the interaction term
    ATTE_estimates.append(twfe)

ATTE_estimates = np.array(ATTE_estimates)

# Convert lists to arrays for calculations
twfe = np.array(twfe)
asymptotic_variance = np.array(asymptotic_variance)

# Calculate metrics
avg_bias = np.mean(twfe - 0)  # Assuming ATTE is 0 as mentioned in the code
med_bias = np.median(twfe - 0)  # Assuming ATTE is 0 as mentioned in the code
rmse = np.sqrt(np.mean((twfe - 0) ** 2))  # Assuming ATTE is 0 as mentioned in the code
variance_ATT = np.var(ATTE_estimates)
coverage_probability = np.mean(
    ((twfe - 1.96 * np.sqrt(variance_ATT)) <= 0)
    & ((twfe + 1.96 * np.sqrt(variance_ATT)) >= 0),
)
avg_ci_length = np.mean(
    2 * 1.96 * np.sqrt(variance_ATT),
)  # Length of 95% confidence interval

# Print results
print(f"Av. Bias: {avg_bias}")
print(f"Med. Bias: {med_bias}")
print(f"RMSE: {rmse}")
print(f"variance_ATT: {variance_ATT}")
print(f"Cover: {coverage_probability}")
print(f"CIL: {avg_ci_length}")

DGP 3

In [ ]:
# dgp panel at work
import numpy as np
import pandas as pd
import statsmodels.api as sm

np.random.seed(42)  # You can use any integer value as the seed

# Sample size
n = 1000
# pscore index (strength of common support)
Xsi_ps = 0.75
# Proportion in each period
_lambda = 0.5
nboot = 199  # Number of bootstrapped draws


# Mean and Std deviation of Z's without truncation
mean_z1 = np.exp(0.25 / 2)
sd_z1 = np.sqrt((np.exp(0.25) - 1) * np.exp(0.25))
mean_z2 = 10
sd_z2 = 0.54164
mean_z3 = 0.21887
sd_z3 = 0.04453
mean_z4 = 402
sd_z4 = 56.63891


# Initialize empty lists to store results
ATTE_estimates = []
asymptotic_variance = []
coverage = []
ci_length = []

for _i in range(1000):
    # Generate covariates
    x1 = np.random.normal(0, 1, n)
    x2 = np.random.normal(0, 1, n)
    x3 = np.random.normal(0, 1, n)
    x4 = np.random.normal(0, 1, n)

    z1 = np.exp(x1 / 2)
    z2 = x2 / (1 + np.exp(x1)) + 10
    z3 = (x1 * x3 / 25 + 0.6) ** 3
    z4 = (x1 + x4 + 20) ** 2

    z1 = (z1 - mean_z1) / sd_z1
    z2 = (z2 - mean_z2) / sd_z2
    z3 = (z3 - mean_z3) / sd_z3
    z4 = (z4 - mean_z4) / sd_z4

    x = np.column_stack((x1, x2, x3, x4))
    z = np.column_stack((z1, z2, z3, z4))

    # Propensity score
    pi = 1 / (1 + np.exp(Xsi_ps * (-z1 + 0.5 * z2 - 0.25 * z3 - 0.1 * z4)))
    d = np.random.uniform(size=n) <= pi

    # Generate aux indexes for the potential outcomes
    index_lin = 210 + 27.4 * x1 + 13.7 * (x2 + x3 + x4)

    # Create heterogenenous effects for the ATT, which is set approximately equal to zero
    index_unobs_het = d * index_lin
    index_att = 0

    # This is the key for consistency of outcome regression
    index_trend = 210 + 27.4 * x1 + 13.7 * (x2 + x3 + x4)
    # v is the unobserved heterogeneity
    v = np.random.normal(index_unobs_het, 1)

    # Gen realized outcome at time 0
    y00 = index_lin + v + np.random.normal(size=n)
    y10 = index_lin + v + np.random.normal(size=n)

    # Gen outcomes at time 1
    y01 = index_lin + v + np.random.normal(size=n) + index_trend  # This is the baseline
    y11 = (
        index_lin + v + np.random.normal(size=n) + index_trend + index_att
    )  # This is the baseline

    # Generate "T"
    ti_nt = 0.5
    ti_t = 0.5
    ti = d * ti_t + (1 - d) * ti_nt
    post = np.random.uniform(size=n) <= ti

    y = np.where(
        d & post,
        y11,
        np.where(~d & post, y01, np.where(~d & ~post, y00, y10)),
    )

    # Gen id
    id_ = np.repeat(np.arange(1, n + 1), 2)
    time = np.tile([0, 1], n)

    # Put in a long data frame
    dta_long = pd.DataFrame(
        {
            "id": id_,
            "time": time,
            "y": np.tile(y, 2),
            "post": np.tile(post.astype(int), 2),
            "d": np.tile(d.astype(int), 2),
            "x1": np.tile(z1, 2),
            "x2": np.tile(z2, 2),
            "x3": np.tile(z3, 2),
            "x4": np.tile(z4, 2),
        },
    )
    dta_long = dta_long.sort_values(["id", "time"])

    # Create interaction term 'post:d'
    dta_long["post:d"] = dta_long["post"] * dta_long["d"]
    # Standard TWFE

    twfe_i = sm.OLS(
        dta_long["y"],
        sm.add_constant(dta_long[["x1", "x2", "x3", "x4", "post", "d", "post:d"]]),
    ).fit()
    twfe = twfe_i.params["post:d"]
    asymptotic_variance.append(twfe_i.cov_params()["post:d"])
    # Convert lists to arrays for calculations
    ATTE_estimates.append(twfe)


# Convert lists to arrays for ease of calculation
ATTE_estimates = np.array(ATTE_estimates)
asymptotic_variance = np.array(asymptotic_variance)
asymptotic_variance = asymptotic_variance.reshape(-1, 8)
asymptotic_variance = asymptotic_variance[:, 0]

# Calculate metrics
avg_bias = np.mean(ATTE_estimates - 0)  # Assuming ATTE is 0 as mentioned in the code
med_bias = np.median(ATTE_estimates - 0)  # Assuming ATTE is 0 as mentioned in the code
rmse = np.sqrt(
    np.mean((ATTE_estimates - 0) ** 2),
)  # Assuming ATTE is 0 as mentioned in the code
variance_ATT = np.var(ATTE_estimates)
coverage_probability = np.mean(
    ((ATTE_estimates - 1.96 * np.sqrt(variance_ATT)) <= 0)
    & ((ATTE_estimates + 1.96 * np.sqrt(variance_ATT)) >= 0),
)
avg_ci_length = np.mean(
    2 * 1.96 * np.sqrt(variance_ATT),
)  # Length of 95% confidence interval
average_ATT_estimate = np.mean(ATTE_estimates)

# Print results
print(f"Average ATT estimate across all runs: {average_ATT_estimate}")

print(f"Av. Bias: {avg_bias}")
print(f"Med. Bias: {med_bias}")
print(f"RMSE: {rmse}")
print(f"variance_ATT: {variance_ATT}")
print(f"Cover: {coverage_probability}")
print(f"CIL: {avg_ci_length}")

DGP 4

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

np.random.seed(42)  # You can use any integer value as the seed
# Sample size
n = 1000

# pscore index (strength of common support)
Xsi_ps = 0.75

# Proportion in each period
lambda_ = 0.5

# Number of bootstrapped draws
nboot = 199

# Mean and Std deviation of Z's without truncation
mean_z1 = np.exp(0.25 / 2)
sd_z1 = np.sqrt((np.exp(0.25) - 1) * np.exp(0.25))
mean_z2 = 10
sd_z2 = 0.54164
mean_z3 = 0.21887
sd_z3 = 0.04453
mean_z4 = 402
sd_z4 = 56.63891


# Initialize empty lists to store results
ATTE_estimates = []
asymptotic_variance = []
coverage = []
ci_length = []

for _i in range(1000):
    # Gen covariates
    x1 = np.random.normal(0, 1, n)
    x2 = np.random.normal(0, 1, n)
    x3 = np.random.normal(0, 1, n)
    x4 = np.random.normal(0, 1, n)

    z1 = np.exp(x1 / 2)
    z2 = x2 / (1 + np.exp(x1)) + 10
    z3 = (x1 * x3 / 25 + 0.6) ** 3
    z4 = (x1 + x4 + 20) ** 2

    z1 = (z1 - mean_z1) / sd_z1
    z2 = (z2 - mean_z2) / sd_z2
    z3 = (z3 - mean_z3) / sd_z3
    z4 = (z4 - mean_z4) / sd_z4

    x = np.column_stack((x1, x2, x3, x4))
    z = np.column_stack((z1, z2, z3, z4))

    # Gen treatment groups
    # Propensity score
    pi = 1 / (1 + np.exp(-Xsi_ps * (-x1 + 0.5 * x2 - 0.25 * x3 - 0.1 * x4)))
    d = np.random.rand(n) <= pi

    # Generate aux indexes for the potential outcomes
    index_lin = 210 + 27.4 * x1 + 13.7 * (x2 + x3 + x4)

    # Create heterogeneous effects for the ATT, which is set approximately equal to zero
    index_unobs_het = d * index_lin
    index_att = 0

    # This is the key for consistency of outcome regression
    index_trend = 210 + 27.4 * x1 + 13.7 * (x2 + x3 + x4)

    # v is the unobserved heterogeneity
    v = np.random.normal(index_unobs_het, 1, n)

    # Gen realized outcome at time 0
    y00 = index_lin + v + np.random.normal(size=n)
    y10 = index_lin + v + np.random.normal(size=n)

    # Gen outcomes at time 1
    # First let's generate potential outcomes: y_1_potential
    y01 = index_lin + v + np.random.normal(size=n) + index_trend
    y11 = index_lin + v + np.random.normal(size=n) + index_trend + index_att

    # Generate "T"
    ti_nt = 0.5
    ti_t = 0.5
    ti = d * ti_t + (1 - d) * ti_nt
    post = np.random.rand(n) <= ti

    # Combine outcomes into panel data format
    y = np.where(
        d & post,
        y11,
        np.where(~d & post, y01, np.where(~d & ~post, y00, y10)),
    )

    # Gen id
    id_ = np.repeat(np.arange(1, n + 1), 2)
    time = np.tile([0, 1], n)

    # Put in a long data frame
    dta_long = pd.DataFrame(
        {
            "id": id_,
            "time": time,
            "y": np.tile(y, 2),
            "post": np.tile(post.astype(int), 2),
            "d": np.tile(d.astype(int), 2),
            "x1": np.tile(z1, 2),
            "x2": np.tile(z2, 2),
            "x3": np.tile(z3, 2),
            "x4": np.tile(z4, 2),
        },
    )

    dta_long = dta_long.sort_values(["id", "time"])
    dta_long["post:d"] = dta_long["post"] * dta_long["d"]

    # Standard TWFE
    twfe_i = sm.OLS(
        dta_long["y"],
        sm.add_constant(dta_long[["x1", "x2", "x3", "x4", "post", "d", "post:d"]]),
    ).fit()
    twfe = twfe_i.params["post:d"]
    asymptotic_variance.append(twfe_i.cov_params()["post:d"])
    ATTE_estimates.append(twfe)

# Convert lists to arrays for calculations

# Convert lists to arrays for ease of calculation
ATTE_estimates = np.array(ATTE_estimates)
asymptotic_variance = np.array(asymptotic_variance)
asymptotic_variance = asymptotic_variance.reshape(-1, 8)
asymptotic_variance = asymptotic_variance[:, 0]


# Calculate metrics
avg_bias = np.mean(ATTE_estimates - 0)  # Assuming ATTE is 0 as mentioned in the code
med_bias = np.median(ATTE_estimates - 0)  # Assuming ATTE is 0 as mentioned in the code
rmse = np.sqrt(
    np.mean((ATTE_estimates - 0) ** 2),
)  # Assuming ATTE is 0 as mentioned in the code
variance_ATT = np.var(ATTE_estimates)
coverage_probability = np.mean(
    ((ATTE_estimates - 1.96 * np.sqrt(variance_ATT)) <= 0)
    & ((ATTE_estimates + 1.96 * np.sqrt(variance_ATT)) >= 0),
)
avg_ci_length = np.mean(
    2 * 1.96 * np.sqrt(variance_ATT),
)  # Length of 95% confidence interval
average_ATT_estimate = np.mean(ATTE_estimates)

# Print results
print(f"Average ATT estimate across all runs: {average_ATT_estimate}")

print(f"Av. Bias: {avg_bias}")
print(f"Med. Bias: {med_bias}")
print(f"RMSE: {rmse}")
print(f"variance_ATT: {variance_ATT}")
print(f"Cover: {coverage_probability}")
print(f"CIL: {avg_ci_length}")

## Deep learning with a few runs doubly robust

In [ ]:
import warnings

import numpy as np
import tensorflow as tf
from doubleml import DoubleMLData, DoubleMLDID
from doubleml.datasets import make_did_SZ2020
from scikeras.wrappers import KerasClassifier  # pip install scikeras
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

warnings.filterwarnings("ignore")
np.random.seed(42)
n_reps = 10  # change that accordingly
n_obs = 1000
ATTE = 0.0

# Enable GPU acceleration if available
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

# Function to create Keras model


def create_model():
    model = Sequential()
    model.add(Dense(32, input_dim=x.shape[1], activation="relu"))  # Simplified
    model.add(Dense(16, activation="relu"))  # Simplified
    model.add(Dense(1, activation="sigmoid"))  # Assuming binary classification
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model


# Initialize arrays to store statistics
biases = np.full(n_reps, np.nan)
variances = np.full(n_reps, np.nan)
rmse_list = np.full(n_reps, np.nan)
coverage = np.full(n_reps, np.nan)
asymptotic_variance = np.full(n_reps, np.nan)
ci_length = np.full(n_reps, np.nan)
ATTE_estimates = np.full(n_reps, np.nan)

early_stopping = EarlyStopping(monitor="loss", patience=2, verbose=0)  # Early stopping

for i_rep in range(n_reps):
    x, y, d = make_did_SZ2020(
        n_obs=n_obs,
        dgp_type=4,
        cross_sectional_data=False,
        return_type="array",
    )
    dml_data = DoubleMLData.from_arrays(x=x, y=y, d=d)

    # Wrap the Keras model with KerasClassifier
    keras_classifier = KerasClassifier(
        build_fn=create_model,
        epochs=3,  # adjust as needed
        batch_size=32,  # Smaller batch size
        verbose=0,
        callbacks=[early_stopping],
    )

    # Use StandardScaler to normalize data and then use the Keras classifier in a pipeline
    ml_m = Pipeline([("scaler", StandardScaler()), ("nn", keras_classifier)])

    # Use LinearRegression for regression
    ml_g = LinearRegression()

    dml_plr = DoubleMLDID(dml_data, ml_g, ml_m)
    dml_plr.fit()

    ATTE_estimates[i_rep] = dml_plr.coef.squeeze()
    confint = dml_plr.confint(level=0.95)
    coverage[i_rep] = (confint["2.5 %"].iloc[0] <= ATTE) & (
        confint["97.5 %"].iloc[0] >= ATTE
    )
    ci_length[i_rep] = confint["97.5 %"].iloc[0] - confint["2.5 %"].iloc[0]
    # Extract standard error from the summary
    summary_df = dml_plr.summary
    std_err = summary_df.loc["d", "std err"]
    asymptotic_variance[i_rep] = std_err**2

# Calculate metrics
avg_bias = np.mean(ATTE_estimates - ATTE)
med_bias = np.median(ATTE_estimates - ATTE)
rmse = np.sqrt(np.mean((ATTE_estimates - ATTE) ** 2))
avg_asymptotic_variance = np.mean(asymptotic_variance)
coverage_probability = np.mean(coverage)
avg_ci_length = np.mean(ci_length)

# Print results
print(f"Av. Bias: {avg_bias}")
print(f"Med. Bias: {med_bias}")
print(f"RMSE: {rmse}")
print(f"Asy. V: {avg_asymptotic_variance}")
print(f"Cover: {coverage_probability}")
print(f"CIL: {avg_ci_length}")

be careful overfitting could be a problem here!!

# Propensity Score IPW Estimating

## Logit run for Propensity score

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from doubleml import DoubleMLData
from doubleml.datasets import make_did_SZ2020
from scipy.stats import iqr, norm

# Define std_ipw_did_rc function


def std_ipw_did_rc(
    y,
    post,
    D,
    covariates=None,
    i_weights=None,
    boot=False,
    boot_type="weighted",
    nboot=None,
    inffunc=False,
):
    # Convert inputs to numpy arrays
    D = np.asarray(D)
    n = len(D)
    y = np.asarray(y)
    post = np.asarray(post)

    # Add constant to covariate vector
    if covariates is None:
        int_cov = np.ones((n, 1))
    else:
        covariates = np.asarray(covariates)
        if np.all(covariates[:, 0] == 1):
            int_cov = covariates
        else:
            int_cov = np.column_stack((np.ones(n), covariates))

    # Weights
    if i_weights is None:
        i_weights = np.ones(n)
    elif np.min(i_weights) < 0:
        msg = "i.weights must be non-negative"
        raise ValueError(msg)
    i_weights = np.asarray(i_weights)

    # Pscore estimation (logit) and fitted values
    PS = sm.Logit(D, int_cov).fit(disp=0)
    ps_fit = PS.predict(int_cov)
    ps_fit = np.clip(ps_fit, 1e-16, 1 - 1e-16)

    # Compute IPW estimator weights
    w_treat_pre = i_weights * D * (1 - post)
    w_treat_post = i_weights * D * post
    w_cont_pre = i_weights * ps_fit * (1 - D) * (1 - post) / (1 - ps_fit)
    w_cont_post = i_weights * ps_fit * (1 - D) * post / (1 - ps_fit)

    # Elements of the influence function (summands)
    eta_treat_pre = w_treat_pre * y / np.mean(w_treat_pre)
    eta_treat_post = w_treat_post * y / np.mean(w_treat_post)
    eta_cont_pre = w_cont_pre * y / np.mean(w_cont_pre)
    eta_cont_post = w_cont_post * y / np.mean(w_cont_post)

    # Estimator of each component
    att_treat_pre = np.mean(eta_treat_pre)
    att_treat_post = np.mean(eta_treat_post)
    att_cont_pre = np.mean(eta_cont_pre)
    att_cont_post = np.mean(eta_cont_post)

    # ATT estimator
    ipw_att = (att_treat_post - att_treat_pre) - (att_cont_post - att_cont_pre)

    # Influence function to compute standard error
    score_ps = i_weights[:, None] * (D - ps_fit)[:, None] * int_cov
    Hessian_ps = np.linalg.inv(np.dot(int_cov.T, score_ps))
    asy_lin_rep_ps = np.dot(score_ps, Hessian_ps)

    # Influence function of the "treat" component
    inf_treat_pre = eta_treat_pre - w_treat_pre * att_treat_pre / np.mean(w_treat_pre)
    inf_treat_post = eta_treat_post - w_treat_post * att_treat_post / np.mean(
        w_treat_post,
    )
    inf_treat = inf_treat_post - inf_treat_pre

    # Influence function of the control component
    inf_cont_pre = eta_cont_pre - w_cont_pre * att_cont_pre / np.mean(w_cont_pre)
    inf_cont_post = eta_cont_post - w_cont_post * att_cont_post / np.mean(w_cont_post)
    inf_cont = inf_cont_post - inf_cont_pre

    # Estimation effect from gamma hat (pscore)
    M2_pre = np.mean(
        w_cont_pre[:, None] * (y - att_cont_pre)[:, None] * int_cov,
        axis=0,
    ) / np.mean(w_cont_pre)
    M2_post = np.mean(
        w_cont_post[:, None] * (y - att_cont_post)[:, None] * int_cov,
        axis=0,
    ) / np.mean(w_cont_post)
    inf_cont_ps = np.dot(asy_lin_rep_ps, (M2_post - M2_pre))

    # Influence function for the control component
    inf_cont += inf_cont_ps

    # Combine influence functions
    att_inf_func = inf_treat - inf_cont

    if not boot:
        # Standard error
        se_att = np.std(att_inf_func) / np.sqrt(n)
        uci = ipw_att + 1.96 * se_att
        lci = ipw_att - 1.96 * se_att
        ipw_boot = None
    else:
        if nboot is None:
            nboot = 999
        if boot_type == "multiplier":
            # Multiplier bootstrap
            ipw_boot = mboot_did(att_inf_func, nboot)
            se_att = iqr(ipw_boot) / (norm.ppf(0.75) - norm.ppf(0.25))
            cv = np.quantile(np.abs(ipw_boot / se_att), 0.95)
            uci = ipw_att + cv * se_att
            lci = ipw_att - cv * se_att
        else:
            # Weighted bootstrap
            ipw_boot = [
                wboot_std_ipw_rc(n, y, post, D, int_cov, i_weights)
                for _ in range(nboot)
            ]
            ipw_boot = np.array(ipw_boot)
            se_att = iqr(ipw_boot - ipw_att) / (norm.ppf(0.75) - norm.ppf(0.25))
            cv = np.quantile(np.abs((ipw_boot - ipw_att) / se_att), 0.95)
            uci = ipw_att + cv * se_att
            lci = ipw_att - cv * se_att

    if not inffunc:
        att_inf_func = None

    return {
        "ATT": ipw_att,
        "se": se_att,
        "uci": uci,
        "lci": lci,
        "boots": ipw_boot,
        "att_inf_func": att_inf_func,
    }


def mboot_did(att_inf_func, nboot):
    n = len(att_inf_func)
    boots = []
    for _ in range(nboot):
        weights = np.random.exponential(scale=1.0, size=n)
        boot_att = np.sum(weights * att_inf_func) / np.sum(weights)
        boots.append(boot_att)
    return np.array(boots)


def wboot_std_ipw_rc(n, y, post, D, int_cov, i_weights):
    indices = np.random.choice(n, n, replace=True)
    y_boot = y[indices]
    post_boot = post[indices]
    D_boot = D[indices]
    int_cov_boot = int_cov[indices]
    i_weights_boot = i_weights[indices]
    return std_ipw_did_rc(
        y_boot,
        post_boot,
        D_boot,
        int_cov_boot,
        i_weights_boot,
        boot=False,
    )["ATT"]


# Simulation with 100 runs
n_runs = 100
true_att = 0  # true average treatment effect

att_estimates = []
se_estimates = []
uci_estimates = []
lci_estimates = []

for _ in range(n_runs):
    x, y, d = make_did_SZ2020(
        n_obs=1000,
        dgp_type=4,
        cross_sectional_data=False,
        return_type="array",
    )
    covariates = x
    post = np.concatenate([np.zeros(500), np.ones(500)])

    results = std_ipw_did_rc(y, post, d, covariates)

    att_estimates.append(results["ATT"])
    se_estimates.append(results["se"])
    uci_estimates.append(results["uci"])
    lci_estimates.append(results["lci"])

att_estimates = np.array(att_estimates)
se_estimates = np.array(se_estimates)
uci_estimates = np.array(uci_estimates)
lci_estimates = np.array(lci_estimates)

# Calculate measures
biases = att_estimates - true_att
avg_bias = np.mean(biases)
med_bias = np.median(biases)
rmse = np.sqrt(np.mean(biases**2))
var_att = np.var(att_estimates)
ci_lengths = uci_estimates - lci_estimates
coverage = np.mean((lci_estimates <= true_att) & (uci_estimates >= true_att))
avg_ci_length = np.mean(
    2 * 1.96 * np.sqrt(var_att),
)  # Length of 95% confidence interval


measures = {
    "Average Bias": avg_bias,
    "Median Bias": med_bias,
    "RMSE": rmse,
    "Variance of ATT": var_att,
    "Coverage": coverage,
    "Confidence Interval Length": avg_ci_length,
}

# Print measures
for measure, value in measures.items():
    print(f"{measure}: {value}")

## just IPW Deep Learning without DR



1:42 min for 100 runs

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from doubleml import DoubleMLData
from doubleml.datasets import make_did_SZ2020
from scipy.stats import iqr, norm
from tensorflow.keras import layers, models
from tensorflow.keras.regularizers import l2

# Define std_ipw_did_rc function


def std_ipw_did_rc(
    y,
    post,
    D,
    covariates=None,
    i_weights=None,
    boot=False,
    boot_type="weighted",
    nboot=None,
    inffunc=False,
):
    # Convert inputs to numpy arrays
    D = np.asarray(D)
    n = len(D)
    y = np.asarray(y)
    post = np.asarray(post)

    # Add constant to covariate vector
    if covariates is None:
        int_cov = np.ones((n, 1))
    else:
        covariates = np.asarray(covariates)
        if np.all(covariates[:, 0] == 1):
            int_cov = covariates
        else:
            int_cov = np.column_stack((np.ones(n), covariates))

    # Weights
    if i_weights is None:
        i_weights = np.ones(n)
    elif np.min(i_weights) < 0:
        msg = "i.weights must be non-negative"
        raise ValueError(msg)
    i_weights = np.asarray(i_weights)

    # Pscore estimation (neural network) and fitted values
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=(int_cov.shape[1],)))
    model.add(
        layers.Dense(10, activation="relu", kernel_regularizer=l2(0.01)),
    )  # Added L2 regularization
    model.add(layers.Dense(1, activation="sigmoid", kernel_regularizer=l2(0.01)))

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )
    model.fit(int_cov, D, epochs=3, batch_size=32, verbose=0)

    ps_fit = model.predict(int_cov).flatten()
    ps_fit = np.clip(ps_fit, 1e-16, 1 - 1e-16)

    # Compute IPW estimator weights
    w_treat_pre = i_weights * D * (1 - post)
    w_treat_post = i_weights * D * post
    w_cont_pre = i_weights * ps_fit * (1 - D) * (1 - post) / (1 - ps_fit)
    w_cont_post = i_weights * ps_fit * (1 - D) * post / (1 - ps_fit)

    # Elements of the influence function (summands)
    eta_treat_pre = w_treat_pre * y / np.mean(w_treat_pre)
    eta_treat_post = w_treat_post * y / np.mean(w_treat_post)
    eta_cont_pre = w_cont_pre * y / np.mean(w_cont_pre)
    eta_cont_post = w_cont_post * y / np.mean(w_cont_post)

    # Estimator of each component
    att_treat_pre = np.mean(eta_treat_pre)
    att_treat_post = np.mean(eta_treat_post)
    att_cont_pre = np.mean(eta_cont_pre)
    att_cont_post = np.mean(eta_cont_post)

    # ATT estimator
    ipw_att = (att_treat_post - att_treat_pre) - (att_cont_post - att_cont_pre)

    # Influence function to compute standard error
    score_ps = i_weights[:, None] * (D - ps_fit)[:, None] * int_cov
    Hessian_ps = np.linalg.inv(np.dot(int_cov.T, score_ps))
    asy_lin_rep_ps = np.dot(score_ps, Hessian_ps)

    # Influence function of the "treat" component
    inf_treat_pre = eta_treat_pre - w_treat_pre * att_treat_pre / np.mean(w_treat_pre)
    inf_treat_post = eta_treat_post - w_treat_post * att_treat_post / np.mean(
        w_treat_post,
    )
    inf_treat = inf_treat_post - inf_treat_pre

    # Influence function of the control component
    inf_cont_pre = eta_cont_pre - w_cont_pre * att_cont_pre / np.mean(w_cont_pre)
    inf_cont_post = eta_cont_post - w_cont_post * att_cont_post / np.mean(w_cont_post)
    inf_cont = inf_cont_post - inf_cont_pre

    # Estimation effect from gamma hat (pscore)
    M2_pre = np.mean(
        w_cont_pre[:, None] * (y - att_cont_pre)[:, None] * int_cov,
        axis=0,
    ) / np.mean(w_cont_pre)
    M2_post = np.mean(
        w_cont_post[:, None] * (y - att_cont_post)[:, None] * int_cov,
        axis=0,
    ) / np.mean(w_cont_post)
    inf_cont_ps = np.dot(asy_lin_rep_ps, (M2_post - M2_pre))

    # Influence function for the control component
    inf_cont += inf_cont_ps

    # Combine influence functions
    att_inf_func = inf_treat - inf_cont

    if not boot:
        # Standard error
        se_att = np.std(att_inf_func) / np.sqrt(n)
        uci = ipw_att + 1.96 * se_att
        lci = ipw_att - 1.96 * se_att
        ipw_boot = None
    else:
        if nboot is None:
            nboot = 999
        if boot_type == "multiplier":
            # Multiplier bootstrap
            ipw_boot = mboot_did(att_inf_func, nboot)
            se_att = iqr(ipw_boot) / (norm.ppf(0.75) - norm.ppf(0.25))
            cv = np.quantile(np.abs(ipw_boot / se_att), 0.95)
            uci = ipw_att + cv * se_att
            lci = ipw_att - cv * se_att
        else:
            # Weighted bootstrap
            ipw_boot = [
                wboot_std_ipw_rc(n, y, post, D, int_cov, i_weights)
                for _ in range(nboot)
            ]
            ipw_boot = np.array(ipw_boot)
            se_att = iqr(ipw_boot - ipw_att) / (norm.ppf(0.75) - norm.ppf(0.25))
            cv = np.quantile(np.abs((ipw_boot - ipw_att) / se_att), 0.95)
            uci = ipw_att + cv * se_att
            lci = ipw_att - cv * se_att

    if not inffunc:
        att_inf_func = None

    return {
        "ATT": ipw_att,
        "se": se_att,
        "uci": uci,
        "lci": lci,
        "boots": ipw_boot,
        "att_inf_func": att_inf_func,
    }


def mboot_did(att_inf_func, nboot):
    n = len(att_inf_func)
    boots = []
    for _ in range(nboot):
        weights = np.random.exponential(scale=1.0, size=n)
        boot_att = np.sum(weights * att_inf_func) / np.sum(weights)
        boots.append(boot_att)
    return np.array(boots)


def wboot_std_ipw_rc(n, y, post, D, int_cov, i_weights):
    indices = np.random.choice(n, n, replace=True)
    y_boot = y[indices]
    post_boot = post[indices]
    D_boot = D[indices]
    int_cov_boot = int_cov[indices]
    i_weights_boot = i_weights[indices]
    return std_ipw_did_rc(
        y_boot,
        post_boot,
        D_boot,
        int_cov_boot,
        i_weights_boot,
        boot=False,
    )["ATT"]


# Simulation with 10 runs
n_runs = 100
true_att = 0  # true average treatment effect

att_estimates = []
se_estimates = []
uci_estimates = []
lci_estimates = []

for _ in range(n_runs):
    x, y, d = make_did_SZ2020(
        n_obs=1000,
        dgp_type=4,
        cross_sectional_data=False,
        return_type="array",
    )
    covariates = x
    post = np.concatenate([np.zeros(500), np.ones(500)])

    results = std_ipw_did_rc(y, post, d, covariates)

    att_estimates.append(results["ATT"])
    se_estimates.append(results["se"])
    uci_estimates.append(results["uci"])
    lci_estimates.append(results["lci"])

att_estimates = np.array(att_estimates)
se_estimates = np.array(se_estimates)
uci_estimates = np.array(uci_estimates)
lci_estimates = np.array(lci_estimates)

# Calculate measures
biases = att_estimates - true_att
avg_bias = np.mean(biases)
med_bias = np.median(biases)
rmse = np.sqrt(np.mean(biases**2))
var_att = np.mean(np.var(att_estimates))
ci_lengths = uci_estimates - lci_estimates
coverage = np.mean((lci_estimates <= true_att) & (uci_estimates >= true_att))
avg_ci_length = np.mean(
    2 * 1.96 * np.sqrt(var_att),
)  # Length of 95% confidence interval

measures = {
    "Average Bias": avg_bias,
    "Median Bias": med_bias,
    "RMSE": rmse,
    "Variance of ATT": var_att,
    "Coverage": coverage,
    "Confidence Interval Length": avg_ci_length,
}

# Print measures
for measure, value in measures.items():
    print(f"{measure}: {value}")

with 3 epochs
Average Bias: -0.7969437934813081
Median Bias: -0.07284544806852011
RMSE: 6.650149057545116
Variance of ATT: 43.58936307759981
Coverage: 0.91
Confidence Interval Length: 25.880718475259332

with 50epochs
Average Bias: 0.1748598802427668
Median Bias: 0.6697785194841401
RMSE: 5.236229617457072
Variance of ATT: 27.38752462901612
Coverage: 0.94
Confidence Interval Length: 20.514571856592898


# try it the code and deep learning and double robust from paper

https://github.com/tommaso-manfe/DR-DIPW-Updated-Beyond-Regression-DiD/blob/main/Simulation%20Github/drdid_imp_rc.R

In [ ]:
# double robust loop with logistic
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression

np.random.seed(42)  # You can use any integer value as the seed


def pscore_cal(D, int_cov, i_weights):
    logit = LogisticRegression()
    logit.fit(int_cov, D, sample_weight=i_weights)
    pscore = logit.predict_proba(int_cov)[:, 1]
    return {"pscore": pscore, "flag": "success"}


def wols_rc(y, post, D, int_cov, ps_fit, i_weights, pre, treat):
    mask = (D == 1) & (post == pre) if treat else (D == 0) & (post == pre)
    model = sm.WLS(y[mask], int_cov[mask], weights=i_weights[mask])
    results = model.fit()
    out_reg = results.predict(int_cov)
    return {"out_reg": out_reg}


def mboot_did(inf_func, nboot):
    n = len(inf_func)
    boot_samples = np.random.choice(inf_func, (nboot, n), replace=True)
    return np.mean(boot_samples, axis=1)


def wboot_drdid_imp_rc(i, n, y, post, D, int_cov, i_weights):
    boot_indices = np.random.choice(np.arange(n), n, replace=True)
    y_boot = y[boot_indices]
    post_boot = post[boot_indices]
    D_boot = D[boot_indices]
    int_cov_boot = int_cov[boot_indices]
    i_weights_boot = i_weights[boot_indices]
    result = drdid_imp_rc(y_boot, post_boot, D_boot, int_cov_boot, i_weights_boot)
    return result["ATT"]


def drdid_imp_rc(
    y,
    post,
    D,
    covariates=None,
    i_weights=None,
    boot=False,
    boot_type="weighted",
    nboot=None,
    inffunc=False,
):
    D = np.asarray(D).flatten()
    n = len(D)
    y = np.asarray(y).flatten()
    post = np.asarray(post).flatten()

    int_cov = np.ones((n, 1))
    if covariates is not None:
        covariates = np.asarray(covariates)
        if np.all(covariates[:, 0] == 1):
            int_cov = covariates
        else:
            int_cov = np.column_stack((np.ones(n), covariates))

    if i_weights is None:
        i_weights = np.ones(n)
    else:
        i_weights = np.asarray(i_weights).flatten()
        if np.min(i_weights) < 0:
            msg = "i.weights must be non-negative"
            raise ValueError(msg)

    # Pscore calculation
    pscore_ipt = pscore_cal(D, int_cov, i_weights)
    ps_fit = np.clip(pscore_ipt["pscore"], 1e-16, 1 - 1e-16)

    # Outcome regression for control group
    out_y_cont_pre = wols_rc(
        y,
        post,
        D,
        int_cov,
        ps_fit,
        i_weights,
        pre=True,
        treat=False,
    )["out_reg"]
    out_y_cont_post = wols_rc(
        y,
        post,
        D,
        int_cov,
        ps_fit,
        i_weights,
        pre=False,
        treat=False,
    )["out_reg"]
    out_y_cont = post * out_y_cont_post + (1 - post) * out_y_cont_pre

    # Outcome regression for treated group using OLS
    reg_treat_coeff_pre = (
        sm.WLS(y, int_cov, weights=i_weights * (D == 1) * (post == 0)).fit().params
    )
    out_y_treat_pre = np.dot(int_cov, reg_treat_coeff_pre)
    reg_treat_coeff_post = (
        sm.WLS(y, int_cov, weights=i_weights * (D == 1) * (post == 1)).fit().params
    )
    out_y_treat_post = np.dot(int_cov, reg_treat_coeff_post)

    # Weights
    w_treat_pre = i_weights * D * (1 - post)
    w_treat_post = i_weights * D * post
    w_cont_pre = i_weights * ps_fit * (1 - D) * (1 - post) / (1 - ps_fit)
    w_cont_post = i_weights * ps_fit * (1 - D) * post / (1 - ps_fit)

    w_d = i_weights * D
    w_dt1 = i_weights * D * post
    w_dt0 = i_weights * D * (1 - post)

    # Influence function elements
    eta_treat_pre = w_treat_pre * (y - out_y_cont) / np.mean(w_treat_pre)
    eta_treat_post = w_treat_post * (y - out_y_cont) / np.mean(w_treat_post)
    eta_cont_pre = w_cont_pre * (y - out_y_cont) / np.mean(w_cont_pre)
    eta_cont_post = w_cont_post * (y - out_y_cont) / np.mean(w_cont_post)

    eta_d_post = w_d * (out_y_treat_post - out_y_cont_post) / np.mean(w_d)
    eta_dt1_post = w_dt1 * (out_y_treat_post - out_y_cont_post) / np.mean(w_dt1)
    eta_d_pre = w_d * (out_y_treat_pre - out_y_cont_pre) / np.mean(w_d)
    eta_dt0_pre = w_dt0 * (out_y_treat_pre - out_y_cont_pre) / np.mean(w_dt0)

    att_treat_pre = np.mean(eta_treat_pre)
    att_treat_post = np.mean(eta_treat_post)
    att_cont_pre = np.mean(eta_cont_pre)
    att_cont_post = np.mean(eta_cont_post)

    att_d_post = np.mean(eta_d_post)
    att_dt1_post = np.mean(eta_dt1_post)
    att_d_pre = np.mean(eta_d_pre)
    att_dt0_pre = np.mean(eta_dt0_pre)

    dr_att = (
        (att_treat_post - att_treat_pre)
        - (att_cont_post - att_cont_pre)
        + (att_d_post - att_dt1_post)
        - (att_d_pre - att_dt0_pre)
    )

    inf_treat_pre = eta_treat_pre - w_treat_pre * att_treat_pre / np.mean(w_treat_pre)
    inf_treat_post = eta_treat_post - w_treat_post * att_treat_post / np.mean(
        w_treat_post,
    )
    inf_treat = inf_treat_post - inf_treat_pre

    inf_cont_pre = eta_cont_pre - w_cont_pre * att_cont_pre / np.mean(w_cont_pre)
    inf_cont_post = eta_cont_post - w_cont_post * att_cont_post / np.mean(w_cont_post)
    inf_cont = inf_cont_post - inf_cont_pre

    dr_att_inf_func1 = inf_treat - inf_cont

    inf_eff1 = eta_d_post - w_d * att_d_post / np.mean(w_d)
    inf_eff2 = eta_dt1_post - w_dt1 * att_dt1_post / np.mean(w_dt1)
    inf_eff3 = eta_d_pre - w_d * att_d_pre / np.mean(w_d)
    inf_eff4 = eta_dt0_pre - w_dt0 * att_dt0_pre / np.mean(w_dt0)
    inf_eff = (inf_eff1 - inf_eff2) - (inf_eff3 - inf_eff4)

    dr_att_inf_func = dr_att_inf_func1 + inf_eff

    if not boot:
        se_dr_att = np.std(dr_att_inf_func) / np.sqrt(n)
        uci = dr_att + 1.96 * se_dr_att
        lci = dr_att - 1.96 * se_dr_att
        dr_boot = None
    else:
        if nboot is None:
            nboot = 999
        if boot_type == "multiplier":
            dr_boot = mboot_did(dr_att_inf_func, nboot)
            se_dr_att = np.subtract(*np.percentile(dr_boot, [75, 25])) / (
                np.percentile(dr_boot, 75) - np.percentile(dr_boot, 25)
            )
            cv = np.percentile(np.abs(dr_boot / se_dr_att), 95)
            uci = dr_att + cv * se_dr_att
            lci = dr_att - cv * se_dr_att
        else:
            dr_boot = np.array(
                [
                    wboot_drdid_imp_rc(i, n, y, post, D, int_cov, i_weights)
                    for i in range(nboot)
                ],
            )
            se_dr_att = np.subtract(*np.percentile(dr_boot - dr_att, [75, 25])) / (
                np.percentile(dr_boot, 75) - np.percentile(dr_boot, 25)
            )
            cv = np.percentile(np.abs((dr_boot - dr_att) / se_dr_att), 95)
            uci = dr_att + cv * se_dr_att
            lci = dr_att - cv * se_dr_att

    if not inffunc:
        dr_att_inf_func = None

    call_param = locals()
    argu = {
        "panel": False,
        "estMethod": "imp",
        "boot": boot,
        "boot.type": boot_type,
        "nboot": nboot,
        "type": "dr",
    }

    return {
        "ATT": dr_att,
        "se": se_dr_att,
        "uci": uci,
        "lci": lci,
        "boots": dr_boot,
        "ps.flag": pscore_ipt["flag"],
        "att.inf.func": dr_att_inf_func,
        "call.param": call_param,
        "argu": argu,
    }


# Simulation with 100 runs
n_runs = 100
true_att = 0  # true average treatment effect

att_estimates = []
se_estimates = []
uci_estimates = []
lci_estimates = []
variances = []

for _ in range(n_runs):
    # Generate synthetic data using make_did_SZ2020 function
    x, y, d = make_did_SZ2020(
        n_obs=1000,
        dgp_type=1,
        cross_sectional_data=False,
        return_type="array",
    )
    covariates = x
    post = np.concatenate([np.zeros(500), np.ones(500)])

    result = drdid_imp_rc(y, post, d, covariates, boot=True, nboot=100)

    att_estimates.append(result["ATT"])
    se_estimates.append(result["se"])
    uci_estimates.append(result["uci"])
    lci_estimates.append(result["lci"])
    variances.append(np.var(result["boots"]))

att_estimates = np.array(att_estimates)
se_estimates = np.array(se_estimates)
uci_estimates = np.array(uci_estimates)
lci_estimates = np.array(lci_estimates)
variances = np.array(variances)

# Calculate measures
biases = att_estimates - true_att
avg_bias = np.mean(biases)
med_bias = np.median(biases)
rmse = np.sqrt(np.mean(biases**2))
mean_variance = np.mean(variances)
ci_lengths = uci_estimates - lci_estimates
coverage = np.mean((lci_estimates <= true_att) & (uci_estimates >= true_att))
avg_ci_length = np.mean(ci_lengths)

measures = {
    "Average Bias": avg_bias,
    "Median Bias": med_bias,
    "RMSE": rmse,
    "Mean Variance of ATT": mean_variance,
    "Coverage": coverage,
    "Confidence Interval Length": avg_ci_length,
}

# Print measures
for measure, value in measures.items():
    print(f"{measure}: {value}")

In [ ]:
# double robust with deep learning

# adjust the neural net use batch_size = 128 and use all over the same layers

import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

np.random.seed(42)  # You can use any integer value as the seed


def pscore_cal(D, int_cov, i_weights):
    # Define the neural network model
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=(int_cov.shape[1],)))
    model.add(
        layers.Dense(10, activation="relu", kernel_regularizer=l2(0.01)),
    )  # Added L2 regularization
    model.add(
        layers.Dense(1, activation="sigmoid", kernel_regularizer=l2(0.01)),
    )  # Added L2 regularization

    # Compile the model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )

    # Train the model
    model.fit(
        int_cov,
        D,
        sample_weight=i_weights,
        epochs=3,
        batch_size=32,
        verbose=0,
    )  # Adjusted epochs

    # Get the propensity scores
    pscore = model.predict(int_cov).flatten()
    return {"pscore": pscore, "flag": "success"}


def wols_rc(y, post, D, int_cov, ps_fit, i_weights, pre, treat):
    mask = (D == 1) & (post == pre) if treat else (D == 0) & (post == pre)
    model = sm.WLS(y[mask], int_cov[mask], weights=i_weights[mask])
    results = model.fit()
    out_reg = results.predict(int_cov)
    return {"out_reg": out_reg}


def mboot_did(inf_func, nboot):
    n = len(inf_func)
    boot_samples = np.random.choice(inf_func, (nboot, n), replace=True)
    return np.mean(boot_samples, axis=1)


def wboot_drdid_imp_rc(i, n, y, post, D, int_cov, i_weights):
    boot_indices = np.random.choice(np.arange(n), n, replace=True)
    y_boot = y[boot_indices]
    post_boot = post[boot_indices]
    D_boot = D[boot_indices]
    int_cov_boot = int_cov[boot_indices]
    i_weights_boot = i_weights[boot_indices]
    result = drdid_imp_rc(y_boot, post_boot, D_boot, int_cov_boot, i_weights_boot)
    return result["ATT"]


def drdid_imp_rc(
    y,
    post,
    D,
    covariates=None,
    i_weights=None,
    boot=False,
    boot_type="weighted",
    nboot=None,
    inffunc=False,
):
    D = np.asarray(D).flatten()
    n = len(D)
    y = np.asarray(y).flatten()
    post = np.asarray(post).flatten()

    int_cov = np.ones((n, 1))
    if covariates is not None:
        covariates = np.asarray(covariates)
        if np.all(covariates[:, 0] == 1):
            int_cov = covariates
        else:
            int_cov = np.column_stack((np.ones(n), covariates))

    if i_weights is None:
        i_weights = np.ones(n)
    else:
        i_weights = np.asarray(i_weights).flatten()
        if np.min(i_weights) < 0:
            msg = "i.weights must be non-negative"
            raise ValueError(msg)

    # Pscore calculation
    pscore_ipt = pscore_cal(D, int_cov, i_weights)
    ps_fit = np.clip(pscore_ipt["pscore"], 1e-16, 1 - 1e-16)

    # Outcome regression for control group
    out_y_cont_pre = wols_rc(
        y,
        post,
        D,
        int_cov,
        ps_fit,
        i_weights,
        pre=True,
        treat=False,
    )["out_reg"]
    out_y_cont_post = wols_rc(
        y,
        post,
        D,
        int_cov,
        ps_fit,
        i_weights,
        pre=False,
        treat=False,
    )["out_reg"]
    out_y_cont = post * out_y_cont_post + (1 - post) * out_y_cont_pre

    # Outcome regression for treated group using OLS
    reg_treat_coeff_pre = (
        sm.WLS(y, int_cov, weights=i_weights * (D == 1) * (post == 0)).fit().params
    )
    out_y_treat_pre = np.dot(int_cov, reg_treat_coeff_pre)
    reg_treat_coeff_post = (
        sm.WLS(y, int_cov, weights=i_weights * (D == 1) * (post == 1)).fit().params
    )
    out_y_treat_post = np.dot(int_cov, reg_treat_coeff_post)

    # Weights
    w_treat_pre = i_weights * D * (1 - post)
    w_treat_post = i_weights * D * post
    w_cont_pre = i_weights * ps_fit * (1 - D) * (1 - post) / (1 - ps_fit)
    w_cont_post = i_weights * ps_fit * (1 - D) * post / (1 - ps_fit)

    w_d = i_weights * D
    w_dt1 = i_weights * D * post
    w_dt0 = i_weights * D * (1 - post)

    # Influence function elements
    eta_treat_pre = w_treat_pre * (y - out_y_cont) / np.mean(w_treat_pre)
    eta_treat_post = w_treat_post * (y - out_y_cont) / np.mean(w_treat_post)
    eta_cont_pre = w_cont_pre * (y - out_y_cont) / np.mean(w_cont_pre)
    eta_cont_post = w_cont_post * (y - out_y_cont) / np.mean(w_cont_post)

    eta_d_post = w_d * (out_y_treat_post - out_y_cont_post) / np.mean(w_d)
    eta_dt1_post = w_dt1 * (out_y_treat_post - out_y_cont_post) / np.mean(w_dt1)
    eta_d_pre = w_d * (out_y_treat_pre - out_y_cont_pre) / np.mean(w_d)
    eta_dt0_pre = w_dt0 * (out_y_treat_pre - out_y_cont_pre) / np.mean(w_dt0)

    att_treat_pre = np.mean(eta_treat_pre)
    att_treat_post = np.mean(eta_treat_post)
    att_cont_pre = np.mean(eta_cont_pre)
    att_cont_post = np.mean(eta_cont_post)

    att_d_post = np.mean(eta_d_post)
    att_dt1_post = np.mean(eta_dt1_post)
    att_d_pre = np.mean(eta_d_pre)
    att_dt0_pre = np.mean(eta_dt0_pre)

    dr_att = (
        (att_treat_post - att_treat_pre)
        - (att_cont_post - att_cont_pre)
        + (att_d_post - att_dt1_post)
        - (att_d_pre - att_dt0_pre)
    )

    inf_treat_pre = eta_treat_pre - w_treat_pre * att_treat_pre / np.mean(w_treat_pre)
    inf_treat_post = eta_treat_post - w_treat_post * att_treat_post / np.mean(
        w_treat_post,
    )
    inf_treat = inf_treat_post - inf_treat_pre

    inf_cont_pre = eta_cont_pre - w_cont_pre * att_cont_pre / np.mean(w_cont_pre)
    inf_cont_post = eta_cont_post - w_cont_post * att_cont_post / np.mean(w_cont_post)
    inf_cont = inf_cont_post - inf_cont_pre

    dr_att_inf_func1 = inf_treat - inf_cont

    inf_eff1 = eta_d_post - w_d * att_d_post / np.mean(w_d)
    inf_eff2 = eta_dt1_post - w_dt1 * att_dt1_post / np.mean(w_dt1)
    inf_eff3 = eta_d_pre - w_d * att_d_pre / np.mean(w_d)
    inf_eff4 = eta_dt0_pre - w_dt0 * att_dt0_pre / np.mean(w_dt0)
    inf_eff = (inf_eff1 - inf_eff2) - (inf_eff3 - inf_eff4)

    dr_att_inf_func = dr_att_inf_func1 + inf_eff

    if not boot:
        se_dr_att = np.std(dr_att_inf_func) / np.sqrt(n)
        uci = dr_att + 1.96 * se_dr_att
        lci = dr_att - 1.96 * se_dr_att
        dr_boot = None
    else:
        if nboot is None:
            nboot = 999
        if boot_type == "multiplier":
            dr_boot = mboot_did(dr_att_inf_func, nboot)
            se_dr_att = np.subtract(*np.percentile(dr_boot, [75, 25])) / (
                np.percentile(dr_boot, 75) - np.percentile(dr_boot, 25)
            )
            cv = np.percentile(np.abs(dr_boot / se_dr_att), 95)
            uci = dr_att + cv * se_dr_att
            lci = dr_att - cv * se_dr_att
        else:
            dr_boot = np.array(
                [
                    wboot_drdid_imp_rc(i, n, y, post, D, int_cov, i_weights)
                    for i in range(nboot)
                ],
            )
            se_dr_att = np.subtract(*np.percentile(dr_boot - dr_att, [75, 25])) / (
                np.percentile(dr_boot, 75) - np.percentile(dr_boot, 25)
            )
            cv = np.percentile(np.abs((dr_boot - dr_att) / se_dr_att), 95)
            uci = dr_att + cv * se_dr_att
            lci = dr_att - cv * se_dr_att

    if not inffunc:
        dr_att_inf_func = None

    call_param = locals()
    argu = {
        "panel": False,
        "estMethod": "imp",
        "boot": boot,
        "boot.type": boot_type,
        "nboot": nboot,
        "type": "dr",
    }

    return {
        "ATT": dr_att,
        "se": se_dr_att,
        "uci": uci,
        "lci": lci,
        "boots": dr_boot,
        "ps.flag": pscore_ipt["flag"],
        "att.inf.func": dr_att_inf_func,
        "call.param": call_param,
        "argu": argu,
    }


# Simulation with 100 runs
n_runs = 100
true_att = 0  # true average treatment effect

att_estimates = []
se_estimates = []
uci_estimates = []
lci_estimates = []
variances = []

for _ in range(n_runs):
    # Generate synthetic data using make_did_SZ2020 function
    x, y, d = make_did_SZ2020(
        n_obs=1000,
        dgp_type=4,
        cross_sectional_data=False,
        return_type="array",
    )
    covariates = x
    post = np.concatenate([np.zeros(500), np.ones(500)])

    result = drdid_imp_rc(y, post, d, covariates, boot=True, nboot=100)

    att_estimates.append(result["ATT"])
    se_estimates.append(result["se"])
    uci_estimates.append(result["uci"])
    lci_estimates.append(result["lci"])
    variances.append(np.var(result["boots"]))

att_estimates = np.array(att_estimates)
se_estimates = np.array(se_estimates)
uci_estimates = np.array(uci_estimates)
lci_estimates = np.array(lci_estimates)
variances = np.array(variances)

# Calculate measures
biases = att_estimates - true_att
avg_bias = np.mean(biases)
med_bias = np.median(biases)
rmse = np.sqrt(np.mean(biases**2))
mean_variance = np.mean(variances)
ci_lengths = uci_estimates - lci_estimates
coverage = np.mean((lci_estimates <= true_att) & (uci_estimates >= true_att))
avg_ci_length = np.mean(ci_lengths)

measures = {
    "Average Bias": avg_bias,
    "Median Bias": med_bias,
    "RMSE": rmse,
    "Variance of ATT": mean_variance,
    "Coverage": coverage,
    "Confidence Interval Length": avg_ci_length,
}

# Print measures
for measure, value in measures.items():
    print(f"{measure}: {value}")

# Visulaization Ideas

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

df_pa = pd.DataFrame(ATTE_estimates, columns=["Estimate"])
g = sns.kdeplot(df_pa, fill=True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Generate random data for four sets of ATTE estimates with different biases
np.random.seed(42)  # For reproducibility
ATTE_estimates1 = np.random.normal(0.018, 0.109, 1000).astype(float)
ATTE_estimates2 = np.random.normal(20, 0.109, 1000).astype(
    float,
)  # Highly biased to the right
ATTE_estimates3 = np.random.normal(-20, 0.109, 1000).astype(
    float,
)  # Highly biased to the left
ATTE_estimates4 = np.random.normal(0.006, 0.109, 1000).astype(float)

# Create dataframes for each set
df_pa1 = pd.DataFrame(ATTE_estimates1, columns=["Estimate"])
df_pa2 = pd.DataFrame(ATTE_estimates2, columns=["Estimate"])
df_pa3 = pd.DataFrame(ATTE_estimates3, columns=["Estimate"])
df_pa4 = pd.DataFrame(ATTE_estimates4, columns=["Estimate"])

# Plot density plots for each set with the true ATT marked at 0 and ensuring 0 is centered
plt.figure(figsize=(15, 10))


def plot_kde(data, subplot, title):
    plt.subplot(subplot)
    sns.kdeplot(data=data, x="Estimate", color="black", linestyle="-")
    plt.axvline(x=0, color="grey", linestyle="--", label="True ATT")
    plt.xlim(-30, 30)  # Ensuring 0 is centered
    plt.title(title)
    plt.legend()


plot_kde(df_pa1, 221, "Density Plot 1")
plot_kde(df_pa2, 222, "Density Plot 2")
plot_kde(df_pa3, 223, "Density Plot 3")
plot_kde(df_pa4, 224, "Density Plot 4")

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Generate random data for four sets of ATTE estimates with different biases
np.random.seed(42)  # For reproducibility
ATTE_estimates1 = np.random.normal(0.018, 0.109, 1000).astype(float)
ATTE_estimates2 = np.random.normal(20, 0.109, 1000).astype(
    float,
)  # Highly biased to the right
ATTE_estimates3 = np.random.normal(-20, 0.109, 1000).astype(
    float,
)  # Highly biased to the left
ATTE_estimates4 = np.random.normal(0.006, 0.109, 1000).astype(float)
ATTE_estimates4 = ATTE_estimates
# Create dataframes for each set
df_pa1 = pd.DataFrame(ATTE_estimates1, columns=["Estimate"])
df_pa2 = pd.DataFrame(ATTE_estimates2, columns=["Estimate"])
df_pa3 = pd.DataFrame(ATTE_estimates3, columns=["Estimate"])
df_pa4 = pd.DataFrame(ATTE_estimates4, columns=["Estimate"])

# Plot density plots for each set with the true ATT marked at 0
plt.figure(figsize=(20, 10))

sns.kdeplot(data=df_pa1, x="Estimate", color="black", linestyle="-", label="Estimate 1")
sns.kdeplot(
    data=df_pa2,
    x="Estimate",
    color="black",
    linestyle="--",
    label="Estimate 2",
)
sns.kdeplot(
    data=df_pa3,
    x="Estimate",
    color="black",
    linestyle="-.",
    label="Estimate 3",
)
sns.kdeplot(data=df_pa4, x="Estimate", color="black", linestyle=":", label="Estimate 4")

plt.axvline(x=0, color="grey", linestyle="--", label="True ATT")
plt.xlim(-25, 25)  # Ensuring 0 is centered
plt.title("Density Plots of ATTE Estimates")
plt.legend()

plt.tight_layout()
plt.show()

# Deep Learning without double robust

check the paper of the phd guys how they did the IPW or essentially the propensity score matching, they recreated Abadie et al. 2005

# the DGPs for cross sectional data

In [ ]:
# DGP1

import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import logistic

np.random.seed(42)  # You can use any integer value as the seed

# Define parameters
n = 1000  # Sample size
Xsi_ps = 0.75  # pscore index
_lambda = 0.5  # Proportion in each period
nboot = 199  # Number of bootstrapped draws

# Define means and standard deviations
mean_z1 = np.exp(0.25 / 2)
sd_z1 = np.sqrt((np.exp(0.25) - 1) * np.exp(0.25))
mean_z2 = 10
sd_z2 = 0.54164
mean_z3 = 0.21887
sd_z3 = 0.04453
mean_z4 = 402
sd_z4 = 56.63891

# Initialize empty lists to store results
ATTE_estimates = []
asymptotic_variance = []
coverage = []
ci_length = []

# Loop for 1000 runs
for _i in range(1000):
    # Generate covariates
    x1 = np.random.normal(0, 1, n)
    x2 = np.random.normal(0, 1, n)
    x3 = np.random.normal(0, 1, n)
    x4 = np.random.normal(0, 1, n)

    z1 = np.exp(x1 / 2)
    z2 = x2 / (1 + np.exp(x1)) + 10
    z3 = (x1 * x3 / 25 + 0.6) ** 3
    z4 = (x1 + x4 + 20) ** 2

    z1 = (z1 - mean_z1) / sd_z1
    z2 = (z2 - mean_z2) / sd_z2
    z3 = (z3 - mean_z3) / sd_z3
    z4 = (z4 - mean_z4) / sd_z4

    x = np.column_stack((x1, x2, x3, x4))
    z = np.column_stack((z1, z2, z3, z4))

    # Propensity score
    pi = logistic.cdf(Xsi_ps * (-z1 + 0.5 * z2 - 0.25 * z3 - 0.1 * z4))
    d = np.random.uniform(size=n) <= pi

    # Generate aux indexes for the potential outcomes
    index_lin = 210 + 27.4 * z1 + 13.7 * (z2 + z3 + z4)
    index_unobs_het = d * (index_lin)
    index_att = 0
    index_trend = 210 + 27.4 * z1 + 13.7 * (z2 + z3 + z4)

    # Generate unobserved heterogeneity
    v = np.random.normal(index_unobs_het, 1)

    # Generate outcomes at time 0 and time 1
    y00 = index_lin + v + np.random.normal(size=n)
    y10 = index_lin + v + np.random.normal(size=n)
    y01 = index_lin + v + np.random.normal(scale=1, size=n) + index_trend
    y11 = index_lin + v + np.random.normal(scale=1, size=n) + index_trend + index_att

    # Generate "T"
    ti_nt = 0.5
    ti_t = 0.5
    ti = d * ti_t + (1 - d) * ti_nt
    post = np.random.uniform(size=n) <= ti

    y = (
        d * post * y11
        + (1 - d) * post * y01
        + (1 - d) * (1 - post) * y00
        + (d) * (1 - post) * y10
    )

    # Generate id
    id_ = np.arange(1, n + 1)

    # Put in a long data frame
    dta_long = pd.DataFrame(
        {
            "id": id_,
            "y": y,
            "post": post.astype(int),
            "d": d.astype(int),
            "x1": z1,
            "x2": z2,
            "x3": z3,
            "x4": z4,
        },
    )
    dta_long["post:d"] = dta_long["post"] * dta_long["d"]

    dta_long = dta_long.sort_values(by="id")

    # Perform TWFE estimation
    twfe_i = sm.OLS(
        dta_long["y"],
        sm.add_constant(dta_long[["x1", "x2", "x3", "x4", "post", "d", "post:d"]]),
    ).fit()
    twfe = twfe_i.params["post:d"]

    # Calculate asymptotic variance
    asymptotic_variance.append(twfe_i.cov_params()["post:d"])

    # Bootstrap for confidence intervals
    # (Note: This part is not included in the provided code, you'll need to implement it separately)

    # Append TWFE estimate to the list
    ATTE_estimates.append(twfe)

# Convert lists to arrays for ease of calculation
ATTE_estimates = np.array(ATTE_estimates)
asymptotic_variance = np.array(asymptotic_variance)
asymptotic_variance = asymptotic_variance.reshape(-1, 8)
asymptotic_variance = asymptotic_variance[:, 0]

# Calculate metrics
avg_bias = np.mean(ATTE_estimates - 0)  # Assuming ATTE is 0 as mentioned in the code
med_bias = np.median(ATTE_estimates - 0)  # Assuming ATTE is 0 as mentioned in the code
rmse = np.sqrt(
    np.mean((ATTE_estimates - 0) ** 2),
)  # Assuming ATTE is 0 as mentioned in the code
avg_asymptotic_variance = np.mean(asymptotic_variance)
coverage_probability = np.mean(
    ((ATTE_estimates - 1.96 * np.sqrt(asymptotic_variance)) <= 0)
    & ((ATTE_estimates + 1.96 * np.sqrt(asymptotic_variance)) >= 0),
)
avg_ci_length = np.mean(
    2 * 1.96 * np.sqrt(asymptotic_variance),
)  # Length of 95% confidence interval
average_ATT_estimate = np.mean(ATTE_estimates)

# Print results
print(f"Average ATT estimate across all runs: {average_ATT_estimate}")

print(f"Av. Bias: {avg_bias}")
print(f"Med. Bias: {med_bias}")
print(f"RMSE: {rmse}")
print(f"Asy. V: {avg_asymptotic_variance}")
print(f"Cover: {coverage_probability}")
print(f"CIL: {avg_ci_length}")

DGP 2

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import logistic

np.random.seed(42)  # You can use any integer value as the seed

# Sample size
n = 1000
# pscore index (strength of common support)
Xsi_ps = 0.75
# Proportion in each period
_lambda = 0.5
# Number of bootstrapped draws
nboot = 199

# Mean and Std deviation of Z's without truncation
mean_z1 = np.exp(0.25 / 2)
sd_z1 = np.sqrt((np.exp(0.25) - 1) * np.exp(0.25))
mean_z2 = 10
sd_z2 = 0.54164
mean_z3 = 0.21887
sd_z3 = 0.04453
mean_z4 = 402
sd_z4 = 56.63891


# Initialize empty lists to store results
ATTE_estimates = []
asymptotic_variance = []
coverage = []
ci_length = []


for _i in range(1000):
    # Generate covariates
    x1 = np.random.normal(0, 1, n)
    x2 = np.random.normal(0, 1, n)
    x3 = np.random.normal(0, 1, n)
    x4 = np.random.normal(0, 1, n)

    z1 = np.exp(x1 / 2)
    z2 = x2 / (1 + np.exp(x1)) + 10
    z3 = (x1 * x3 / 25 + 0.6) ** 3
    z4 = (x1 + x4 + 20) ** 2

    z1 = (z1 - mean_z1) / sd_z1
    z2 = (z2 - mean_z2) / sd_z2
    z3 = (z3 - mean_z3) / sd_z3
    z4 = (z4 - mean_z4) / sd_z4

    x = np.column_stack((x1, x2, x3, x4))
    z = np.column_stack((z1, z2, z3, z4))

    # Generate treatment groups
    # Propensity score
    pi = logistic.cdf(Xsi_ps * (-x1 + 0.5 * x2 - 0.25 * x3 - 0.1 * x4))
    d = np.random.uniform(size=n) <= pi

    # Generate aux indexes for the potential outcomes
    index_lin = 210 + 27.4 * z1 + 13.7 * (z2 + z3 + z4)

    # Create heterogenenous effects for the ATT, which is set approximately equal to zero
    index_unobs_het = d * (index_lin)
    index_att = 0

    # This is the key for consistency of outcome regression
    index_trend = 210 + 27.4 * z1 + 13.7 * (z2 + z3 + z4)

    # v is the unobserved heterogeneity
    v = np.random.normal(index_unobs_het, 1)

    # Gen realized outcome at time 0
    y00 = index_lin + v + np.random.normal(size=n)
    y10 = index_lin + v + np.random.normal(size=n)

    # Gen outcomes at time 1
    # First let's generate potential outcomes: y_1_potential
    y01 = (
        index_lin + v + np.random.normal(scale=1, size=n) + index_trend
    )  # This is the baseline
    y11 = (
        index_lin + v + np.random.normal(scale=1, size=n) + index_trend + index_att
    )  # This is the baseline

    # Generate "T"
    ti_nt = 0.5
    ti_t = 0.5
    ti = d * ti_t + (1 - d) * ti_nt
    post = np.random.uniform(size=n) <= ti

    y = (
        d * post * y11
        + (1 - d) * post * y01
        + (1 - d) * (1 - post) * y00
        + (d) * (1 - post) * y10
    )

    # Gen id
    id_ = np.arange(1, n + 1)

    # Put in a long data frame
    dta_long = pd.DataFrame(
        {
            "id": id_,
            "y": y,
            "post": post.astype(int),
            "d": d.astype(int),
            "x1": z1,
            "x2": z2,
            "x3": z3,
            "x4": z4,
        },
    )
    dta_long = dta_long.sort_values(by="id")

    # Create interaction term 'post:d'
    dta_long["post:d"] = dta_long["post"] * dta_long["d"]

    twfe_i = sm.OLS(
        dta_long["y"],
        sm.add_constant(dta_long[["x1", "x2", "x3", "x4", "post", "d", "post:d"]]),
    ).fit()
    twfe = twfe_i.params["post:d"]

    asymptotic_variance.append(
        twfe_i.HC0_se["post:d"],
    )  # Append the standard error for the interaction term


# Convert lists to arrays for calculations
twfe = np.array(twfe)
asymptotic_variance = np.array(asymptotic_variance)

# Calculate metrics
avg_bias = np.mean(twfe - 0)  # Assuming ATTE is 0 as mentioned in the code
med_bias = np.median(twfe - 0)  # Assuming ATTE is 0 as mentioned in the code
rmse = np.sqrt(np.mean((twfe - 0) ** 2))  # Assuming ATTE is 0 as mentioned in the code
avg_asymptotic_variance = np.mean(asymptotic_variance)
coverage_probability = np.mean(
    ((twfe - 1.96 * np.sqrt(asymptotic_variance)) <= 0)
    & ((twfe + 1.96 * np.sqrt(asymptotic_variance)) >= 0),
)
avg_ci_length = np.mean(
    2 * 1.96 * np.sqrt(asymptotic_variance),
)  # Length of 95% confidence interval

# Print results
print(f"Av. Bias: {avg_bias}")
print(f"Med. Bias: {med_bias}")
print(f"RMSE: {rmse}")
print(f"Asy. V: {avg_asymptotic_variance}")
print(f"Cover: {coverage_probability}")
print(f"CIL: {avg_ci_length}")

DGP 3

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

np.random.seed(42)  # You can use any integer value as the seed

# Sample size
n = 1000
# pscore index (strength of common support)
Xsi_ps = 0.75
# Proportion in each period
_lambda = 0.5
nboot = 199  # Number of bootstrapped draws


# Mean and Std deviation of Z's without truncation
mean_z1 = np.exp(0.25 / 2)
sd_z1 = np.sqrt((np.exp(0.25) - 1) * np.exp(0.25))
mean_z2 = 10
sd_z2 = 0.54164
mean_z3 = 0.21887
sd_z3 = 0.04453
mean_z4 = 402
sd_z4 = 56.63891


# Initialize empty lists to store results
ATTE_estimates = []
asymptotic_variance = []
coverage = []
ci_length = []

for _i in range(1000):
    # Generate covariates
    x1 = np.random.normal(0, 1, n)
    x2 = np.random.normal(0, 1, n)
    x3 = np.random.normal(0, 1, n)
    x4 = np.random.normal(0, 1, n)

    z1 = np.exp(x1 / 2)
    z2 = x2 / (1 + np.exp(x1)) + 10
    z3 = (x1 * x3 / 25 + 0.6) ** 3
    z4 = (x1 + x4 + 20) ** 2

    z1 = (z1 - mean_z1) / sd_z1
    z2 = (z2 - mean_z2) / sd_z2
    z3 = (z3 - mean_z3) / sd_z3
    z4 = (z4 - mean_z4) / sd_z4

    x = np.column_stack((x1, x2, x3, x4))
    z = np.column_stack((z1, z2, z3, z4))

    # Propensity score
    pi = 1 / (1 + np.exp(Xsi_ps * (-z1 + 0.5 * z2 - 0.25 * z3 - 0.1 * z4)))
    d = np.random.uniform(size=n) <= pi

    # Generate aux indexes for the potential outcomes
    index_lin = 210 + 27.4 * x1 + 13.7 * (x2 + x3 + x4)

    # Create heterogenenous effects for the ATT, which is set approximately equal to zero
    index_unobs_het = d * index_lin
    index_att = 0

    # This is the key for consistency of outcome regression
    index_trend = 210 + 27.4 * x1 + 13.7 * (x2 + x3 + x4)
    # v is the unobserved heterogeneity
    v = np.random.normal(index_unobs_het, 1)

    # Gen realized outcome at time 0
    y00 = index_lin + v + np.random.normal(size=n)
    y10 = index_lin + v + np.random.normal(size=n)

    # Gen outcomes at time 1
    y01 = index_lin + v + np.random.normal(size=n) + index_trend  # This is the baseline
    y11 = (
        index_lin + v + np.random.normal(size=n) + index_trend + index_att
    )  # This is the baseline

    # Generate "T"
    ti_nt = 0.5
    ti_t = 0.5
    ti = d * ti_t + (1 - d) * ti_nt
    post = np.random.uniform(size=n) <= ti

    y = (
        d * post * y11
        + (1 - d) * post * y01
        + (1 - d) * (1 - post) * y00
        + (d) * (1 - post) * y10
    )

    # Gen id
    id_ = np.arange(1, n + 1)

    # Put in a long data frame
    dta_long = pd.DataFrame(
        {
            "id": id_,
            "y": y,
            "post": post.astype(int),
            "d": d.astype(int),
            "x1": z1,
            "x2": z2,
            "x3": z3,
            "x4": z4,
        },
    )
    dta_long = dta_long.sort_values(by="id")
    # Create interaction term 'post:d'
    dta_long["post:d"] = dta_long["post"] * dta_long["d"]
    # Standard TWFE

    twfe_i = sm.OLS(
        dta_long["y"],
        sm.add_constant(dta_long[["x1", "x2", "x3", "x4", "post", "d", "post:d"]]),
    ).fit()
    twfe = twfe_i.params["post:d"]
    asymptotic_variance.append(twfe_i.cov_params()["post:d"])
    # Convert lists to arrays for calculations
    ATTE_estimates.append(twfe)


# Convert lists to arrays for ease of calculation
ATTE_estimates = np.array(ATTE_estimates)
asymptotic_variance = np.array(asymptotic_variance)
asymptotic_variance = asymptotic_variance.reshape(-1, 8)
asymptotic_variance = asymptotic_variance[:, 0]

# Calculate metrics
avg_bias = np.mean(ATTE_estimates - 0)  # Assuming ATTE is 0 as mentioned in the code
med_bias = np.median(ATTE_estimates - 0)  # Assuming ATTE is 0 as mentioned in the code
rmse = np.sqrt(
    np.mean((ATTE_estimates - 0) ** 2),
)  # Assuming ATTE is 0 as mentioned in the code
avg_asymptotic_variance = np.mean(asymptotic_variance)
coverage_probability = np.mean(
    ((ATTE_estimates - 1.96 * np.sqrt(asymptotic_variance)) <= 0)
    & ((ATTE_estimates + 1.96 * np.sqrt(asymptotic_variance)) >= 0),
)
avg_ci_length = np.mean(
    2 * 1.96 * np.sqrt(asymptotic_variance),
)  # Length of 95% confidence interval
average_ATT_estimate = np.mean(ATTE_estimates)

# Print results
print(f"Average ATT estimate across all runs: {average_ATT_estimate}")

print(f"Av. Bias: {avg_bias}")
print(f"Med. Bias: {med_bias}")
print(f"RMSE: {rmse}")
print(f"Asy. V: {avg_asymptotic_variance}")
print(f"Cover: {coverage_probability}")
print(f"CIL: {avg_ci_length}")

DGP 4 cross sectional

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

np.random.seed(42)  # You can use any integer value as the seed
# Sample size
n = 1000

# pscore index (strength of common support)
Xsi_ps = 0.75

# Proportion in each period
lambda_ = 0.5

# Number of bootstrapped draws
nboot = 199

# Mean and Std deviation of Z's without truncation
mean_z1 = np.exp(0.25 / 2)
sd_z1 = np.sqrt((np.exp(0.25) - 1) * np.exp(0.25))
mean_z2 = 10
sd_z2 = 0.54164
mean_z3 = 0.21887
sd_z3 = 0.04453
mean_z4 = 402
sd_z4 = 56.63891


# Initialize empty lists to store results
ATTE_estimates = []
asymptotic_variance = []
coverage = []
ci_length = []


for _i in range(1000):
    # Gen covariates
    x1 = np.random.normal(0, 1, n)
    x2 = np.random.normal(0, 1, n)
    x3 = np.random.normal(0, 1, n)
    x4 = np.random.normal(0, 1, n)

    z1 = np.exp(x1 / 2)
    z2 = x2 / (1 + np.exp(x1)) + 10
    z3 = (x1 * x3 / 25 + 0.6) ** 3
    z4 = (x1 + x4 + 20) ** 2

    z1 = (z1 - mean_z1) / sd_z1
    z2 = (z2 - mean_z2) / sd_z2
    z3 = (z3 - mean_z3) / sd_z3
    z4 = (z4 - mean_z4) / sd_z4

    x = np.column_stack((x1, x2, x3, x4))
    z = np.column_stack((z1, z2, z3, z4))

    # Gen treatment groups
    # Propensity score
    pi = 1 / (1 + np.exp(-Xsi_ps * (-x1 + 0.5 * x2 - 0.25 * x3 - 0.1 * x4)))
    d = np.random.rand(n) <= pi

    # Generate aux indexes for the potential outcomes
    index_lin = 210 + 27.4 * x1 + 13.7 * (x2 + x3 + x4)

    # Create heterogeneous effects for the ATT, which is set approximately equal to zero
    index_unobs_het = d * index_lin
    index_att = 0

    # This is the key for consistency of outcome regression
    index_trend = 210 + 27.4 * x1 + 13.7 * (x2 + x3 + x4)

    # v is the unobserved heterogeneity
    v = np.random.normal(index_unobs_het, 1, n)

    # Gen realized outcome at time 0
    y00 = index_lin + v + np.random.normal(size=n)
    y10 = index_lin + v + np.random.normal(size=n)

    # Gen outcomes at time 1
    # First let's generate potential outcomes: y_1_potential
    y01 = index_lin + v + np.random.normal(size=n) + index_trend
    y11 = index_lin + v + np.random.normal(size=n) + index_trend + index_att

    # Generate "T"
    ti_nt = 0.5
    ti_t = 0.5
    ti = d * ti_t + (1 - d) * ti_nt
    post = np.random.rand(n) <= ti

    y = (
        d * post * y11
        + (1 - d) * post * y01
        + (1 - d) * (1 - post) * y00
        + d * (1 - post) * y10
    )

    # Gen id
    id_ = np.arange(1, n + 1)

    # Put in a long data frame
    dta_long = pd.DataFrame(
        {
            "id": id_,
            "y": y,
            "post": post.astype(int),
            "d": d.astype(int),
            "x1": z1,
            "x2": z2,
            "x3": z3,
            "x4": z4,
        },
    )

    dta_long = dta_long.sort_values("id")
    dta_long["post:d"] = dta_long["post"] * dta_long["d"]
    # Standard TWFE
    twfe_i = sm.OLS(
        dta_long["y"],
        sm.add_constant(dta_long[["x1", "x2", "x3", "x4", "post", "d", "post:d"]]),
    ).fit()
    twfe = twfe_i.params["post:d"]
    asymptotic_variance.append(twfe_i.cov_params()["post:d"])
    ATTE_estimates.append(twfe)
# Convert lists to arrays for calculations

# Calculate metrics

# Convert lists to arrays for ease of calculation
ATTE_estimates = np.array(ATTE_estimates)
asymptotic_variance = np.array(asymptotic_variance)
asymptotic_variance = asymptotic_variance.reshape(-1, 8)
asymptotic_variance = asymptotic_variance[:, 0]

# Calculate metrics
avg_bias = np.mean(ATTE_estimates - 0)  # Assuming ATTE is 0 as mentioned in the code
med_bias = np.median(ATTE_estimates - 0)  # Assuming ATTE is 0 as mentioned in the code
rmse = np.sqrt(
    np.mean((ATTE_estimates - 0) ** 2),
)  # Assuming ATTE is 0 as mentioned in the code
avg_asymptotic_variance = np.mean(asymptotic_variance)
coverage_probability = np.mean(
    ((ATTE_estimates - 1.96 * np.sqrt(asymptotic_variance)) <= 0)
    & ((ATTE_estimates + 1.96 * np.sqrt(asymptotic_variance)) >= 0),
)
avg_ci_length = np.mean(
    2 * 1.96 * np.sqrt(asymptotic_variance),
)  # Length of 95% confidence interval
average_ATT_estimate = np.mean(ATTE_estimates)

# Print results
print(f"Average ATT estimate across all runs: {average_ATT_estimate}")

print(f"Av. Bias: {avg_bias}")
print(f"Med. Bias: {med_bias}")
print(f"RMSE: {rmse}")
print(f"Asy. V: {avg_asymptotic_variance}")
print(f"Cover: {coverage_probability}")
print(f"CIL: {avg_ci_length}")